In [93]:
# imports
import os

import numpy as np  
import pandas as pd 

import sklearn
import time

from matplotlib import pyplot as plt

from surprise import ( 
  Reader,
  Dataset,
  NormalPredictor,
  KNNBasic,
  KNNWithMeans,
  KNNBaseline,
  KNNWithZScore,
  SVD,
  BaselineOnly,
  SVDpp,
  NMF,
  SlopeOne,
  CoClustering,
  accuracy
)

from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import AlgoBase
from sklearn.metrics.pairwise import sigmoid_kernel


In [2]:
# from exploratory_data_analysis import load_set

current = os.getcwd () [ 0 : os.getcwd ().rfind( '\\' ) ]

# movielens dataset path
DATA_PATH = current + '\\dataset\\data.csv'
ITEM_PATH = current + '\\dataset\\item.csv'
USER_PATH = current + '\\dataset\\user.csv'

def load_set (name: str) -> pd.DataFrame: 

  if name == 'DATA':
    columns = [ 'userID', 'itemID', 'rating', 'timestamp' ]
    df = pd.read_csv ( 
      DATA_PATH, 
      names=columns, 
      sep='\t', 
      encoding='latin-1', 
      skipinitialspace=True 
    )
    # df.drop ( columns= [ 'timestamp' ] )
    return df
  
  if name == 'USER':
    columns = [ 'userID', 'age', 'gender', 'occupation', 'zipCode' ]
    df = pd.read_csv ( 
      USER_PATH, 
      names=columns, 
      sep='|', 
      encoding='latin-1', 
      skipinitialspace=True 
    )
    # df.drop ( columns= [ 'zipCode' ] )
    return df
  
  if name == 'ITEM':
    columns = [ 
      'itemID', 
      'name', 
      'releaseDate', 
      'videoReleaseDate', 
      'IMDbURL', 
      'gender_unknown', 
      'gender_action', 
      'gender_adventure', 
      'gender_animation', 
      'gender_children', 
      'gender_comedy',
      'gender_crime',
      'gender_documentary',
      'gender_drama',
      'gender_fantasy',
      'gender_film_noir',
      'gender_horror',
      'gender_musical',
      'gender_mystery',
      'gender_romance',
      'gender_scifi',
      'gender_thriller',
      'gender_war',
      'gender_western',
    ]
    df = pd.read_csv ( 
      ITEM_PATH, 
      names=columns, 
      sep='|', 
      encoding='latin-1', 
      skipinitialspace=True 
    )
    # df.drop ( columns= [ 'zipCode' ] )
    return df


In [48]:
df_data = load_set ( 'DATA' )
df_item = load_set ( 'ITEM' )
df_user = load_set ( 'USER' )

print ( f"""SHAPES
DATA:      { df_data.shape }
ITEM:      { df_item.shape }
USER:      { df_user.shape }
""" )

SHAPES
DATA:      (100000, 4)
ITEM:      (1682, 24)
USER:      (943, 5)



## Model Selection 

In [51]:
reader = Reader ( rating_scale= (1,5) )
data = Dataset.load_from_df ( df_data [ [ 'userID', 'itemID', 'rating' ] ], reader )

In [52]:
benchmark = { }
algorithms = { 
  'SVD' : SVD(),
  'SVD++' : SVDpp(),
  'Slope One': SlopeOne(),
  'NMF': NMF(),
  'Normal Predictor': NormalPredictor(),
  'KNN Baseline': KNNBaseline(),
  'KNN with Means': KNNWithMeans(),
  'KNN Basic': KNNBasic(),
  'KNN with ZScore': KNNWithZScore(),
  'Baseline Only': BaselineOnly(),
  'CoClustering': CoClustering()
}

In [53]:
marks = {  
  'test_rmse' : 'Test RMSE',
  'fit_time'  : 'Fit Time',
  'test_time' : 'Test Time'
}
for algorithm in algorithms.keys():
  results = cross_validate ( algorithms[algorithm], data, measures= ['RMSE'], cv=5, verbose=False )
  tmp = { }
  for key in marks.keys():
    tmp [ marks[key] ] = pd.Series ( results [ key ] ).mean()

  benchmark [ algorithm ] = tmp

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

In [54]:
surprise_results = pd.DataFrame.from_dict ( benchmark ).T
surprise_results

,Test RMSE,Fit Time,Test Time
SVD,0.936049,0.814492,0.100123
SVD++,0.919699,17.490100,2.663680
Slope One,0.944871,0.526992,1.585960
NMF,0.962602,1.446754,0.061940
Normal Predictor,1.517946,0.080726,0.046327
KNN Baseline,0.930667,0.571479,2.132689
KNN with Means,0.950019,0.426015,1.992534
KNN Basic,0.978846,0.400577,1.887710
KNN with ZScore,0.950872,0.482185,2.070479
Baseline Only,0.944120,0.185536,0.068450


In [55]:
surprise_results.sort_values( 'Test RMSE' )

,Test RMSE,Fit Time,Test Time
SVD++,0.919699,17.490100,2.663680
KNN Baseline,0.930667,0.571479,2.132689
SVD,0.936049,0.814492,0.100123
Baseline Only,0.944120,0.185536,0.068450
Slope One,0.944871,0.526992,1.585960
KNN with Means,0.950019,0.426015,1.992534
KNN with ZScore,0.950872,0.482185,2.070479
NMF,0.962602,1.446754,0.061940
CoClustering,0.964294,1.712833,0.055470
KNN Basic,0.978846,0.400577,1.887710


Aquí tenemos una situación, usando todos estos modelos tenemos que la diferencia de RMSE es baja para casi la mayoría, excepto Normal Predictor. Aquí el que menos error tiene es SVD++, pero tiene se demora mucho más que la otra mayoría de los modelos. Por tanto se usará el modelo de KNN Baseline

In [96]:
bsl_options = {
  'method' : 'sgd',
  'learning_rate' : .005,
  'n_epochs' : 100,
  'reg' : 0.02
}

In [91]:
bsl_options = {
  'method' : 'als'
}

In [95]:
trainset, testset = train_test_split ( data, test_size=0.25 )

knn_baseline = KNNBaseline ( bsl_options=bsl_options )
fit = knn_baseline.fit ( trainset )
predictions = fit.test ( testset )
accuracy.rmse ( predictions )

Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9370


0.9370355519717466

A `baseline` es un punto de referencia fijo que se utiliza con fines comparativos. Y estos pueden ser estimados en dos formas diferentes:

- Stochastic Gradient Descent (SGD)
- Alternating Least Squares (ALS)

Anteriormente se muestra el entrenamiento de un modelo usando diferentes configuraciones de ambas formas. Los mejores resultados que se obtuvieron fueron los siguientes:

**SGD** $\to$ RMSE: 0.9335325282503345
- Learning Rate: .005
- No. Epochs: 100
- Reg: 0.02

**ALS** $\to$ RMSE: 0.931172493299744

> Configuración por default de ALS


## Enfoque Híbrido con Surprise

La idea tras modelos híbridos es tomar diferentes sistemas de recomendación y lograr crear combinaciones que alguno de los sistema logre opacar la desventaja que traen los otros sistemas de recomendación. A continuación se presenta diferentes combinaciones de modelos, que provee Surprise, para lograr crear un sistema híbrido que tenga un valor bajo de RMSE y MAE, además que sea capaz de solucionar el problema de los sistemas de recomendación basados en filtrado colaborativo: Cold-Start

> **Pregunta:** ¿En qué consiste Cold-Start?
> 
> El problema cold-start en los sistemas de recomendación basados en el filtrado colaborativo se refiere a la dificultad que surge cuando un sistema de recomendación debe recomendar contenido a nuevos usuarios o productos que no tienen suficientes datos históricos para realizar predicciones precisas.

La implementación de nuestro sistema de recomendación se conformará en dos fases:
- Uso de un LLM como Gemini para que interactue con el nuevo usuario para saber las películas que vió y los géneros que le gustan para extraer los Features del nuevo usuario  
- Extraido los Features del nuevo usuario encontrar las P personas más cercanas que tengan los mismos Features, y luego mediante el modelo híbrido obtener los K items para cada persona, luego aplicamos un 'merge' a estos resultados y ordenamos para obtener unos nuevos K items para el usuario nuevo.  

La idea tras esta implementación es que sea capaz nuestro sistema de recomendación al presentar un nuevo usuario ser capaz de establecer una conversación con este para saber sus gustos y las películas que vió. Luego coger esta información y pasarla a un algoritmo para obtener las personas más cercanas. Seguido aplicar el algoritmo de predicción del modelo híbrido para obtener K items por cada persona cercana al nuevo usuario. Esto permitirá tener un poco más de información sobre las películas que pueden gustarle dada la conversación con tuvo con el LLM y permite lograr además que ese punto negativo que tenian los sistemas de recomendación presentan, principalmente los de filtrado colaborativo, desaparezca un poco

Esta implementación no elimina que haya películas que al usuario le gustarían más y nuestro sistema no lo recomienda, pero es una posible solución a lo que queríamos lograr: solución de cold-start y que las predicciones tengan un bajo valor de las métricas de error de modelos de Machine Learning

In [97]:
class HybridMode ( AlgoBase ):
  def __init__(self, svd_model, knn_model, **kwargs):
    super().__init__(**kwargs)
    self.svd_model = svd_model
    self.knn_model = knn_model
  
  def fit ( self, trainset ): 
    AlgoBase.fit ( self, trainset )
    self.svd_model.fit ( trainset )
    self.knn_model.fit ( trainset )
  
  def estimate ( self, u, i ):
    svd_prediction = self.svd_model.predict ( u, i ).est
    knn_prediction = self.knn_model.predict ( u, i ).est

    return ( svd_prediction + knn_prediction ) / 2

In [98]:
trainset, testset = train_test_split ( data, test_size=0.25 )
svd = SVD () 
knn = KNNWithMeans ( sim_options= { 'name': 'cosine', 'user_based': False } )

hybrid = HybridMode( svd, knn )
hybrid.fit ( trainset )
predictions = hybrid.test ( testset )

rmse = accuracy.rmse ( predictions, verbose=False )
print ( f'RMSE: { rmse }' )

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2929670580796637
